In [1]:
import os

In [2]:
%pwd

'c:\\Users\\suraj\\Desktop\\CODING\\Face-Classification-CNN\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\suraj\\Desktop\\CODING\\Face-Classification-CNN'

In [5]:
import tensorflow as tf

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict
    params_image_size : list
    params_batch_size : int

In [14]:
from  src.FaceClassification.constants import *
from src.FaceClassification.utils.common import read_yaml, create_directories, save_json

In [21]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = Path("artifacts/training/model.keras"),
            training_data = Path("artifacts/data_ingestion/All-images"),
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config

In [19]:
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear" 
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            seed = 42,
            class_mode = "categorical",
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path = Path("scores.json"), data = scores)



In [24]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2025-10-23 22:40:28,045: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-23 22:40:28,048: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-23 22:40:28,050: INFO: common: Created directory at: artifacts]
Found 16 images belonging to 2 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.8125 - loss: 0.3683
[2025-10-23 22:40:33,486: INFO: common: json file saved at: scores.json]
